In [1]:
from map_manager.map_generator import map
from map_manager.noise import toddler
from map_manager.world_encounters import *
from genetic_algorithm.crossover import * 
from genetic_algorithm.mutation import * 
import pickle

In [2]:
nw = map()

nw.populate_map(n_locations=1024, name="terrain",relaxed=True,k=100)
nw.populate_map(n_locations=1024, name="civilisation",relaxed=True,k=100)
nw.populate_map(n_locations=1024, name="threat",relaxed=True, k=100)
nw.populate_map(n_locations=1024, name="faction",relaxed=True, k=100)

nw.attribute_view(seed_1=20,seed_2=30,map_name="terrain",view_name="terrain")
nw.attribute_view(seed_1=40,seed_2=50,map_name="civilisation",view_name="civilisation")
nw.attribute_view(seed_1=34,seed_2=34,map_name="threat",view_name="threat",double=False)
nw.attribute_view(seed_1=67,seed_2=89,map_name="faction",view_name="faction",double=True)

picklefile = open('nw', 'wb')
pickle.dump(nw, picklefile)
picklefile.close()

with open("nw","rb") as f:
    nw = pickle.load(f)

/Users/cetiners/Desktop/Thesis/human_error/map_manager/map_attributer.py:84: RuntimeWarning: invalid value encountered in true_divide
  average = sum_/count


In [3]:
nw.land_mask()

In [4]:
pop = pack_population(nw,pop_size=100,type="camel",pack_size=50,encounter_type="friendly_animals")

pop.evolve(200,mu_p=0.40,save=True)


Gen 1 evolving:
Best Individual: -244000

Gen 2 evolving:
Best Individual: -233000

Gen 3 evolving:
Best Individual: -226000

Gen 4 evolving:
Best Individual: -203000

Gen 5 evolving:
Best Individual: -202000

Gen 6 evolving:
Best Individual: -204000

Gen 7 evolving:
Best Individual: -194000

Gen 8 evolving:
Best Individual: -168000

Gen 9 evolving:
Best Individual: -181000

Gen 10 evolving:
Best Individual: -154000

Gen 11 evolving:
Best Individual: -161000

Gen 12 evolving:
Best Individual: -152000

Gen 13 evolving:
Best Individual: -152000

Gen 14 evolving:
Best Individual: -145000

Gen 15 evolving:
Best Individual: -132000

Gen 16 evolving:
Best Individual: -131000

Gen 17 evolving:
Best Individual: -123000

Gen 18 evolving:
Best Individual: -115000

Gen 19 evolving:
Best Individual: -115000

Gen 20 evolving:
Best Individual: -108000

Gen 21 evolving:
Best Individual: -108000

Gen 22 evolving:
Best Individual: -93000

Gen 23 evolving:
Best Individual: -93000

Gen 24 evolving:
Best

In [ ]:
a = pop.best_inds["200"].pack_coords

civ = nw.views["civilisation"].copy()
ter = nw.views["terrain"].copy()
thr = nw.views["threat"].copy()
fac = nw.views["faction"].copy()

for i in range(1024):
    for j in range(1024):
        civ[i,j] += 1
        ter[i,j] += 1
        thr[i,j] += 1
        fac[i,j] += 1

import matplotlib.pyplot as plt
import numpy as np 

fig, ax = plt.subplots(1,4)
fig.set_dpi(150)
fig.set_size_inches(18, 6)

import random
seed=random.randint(1,100)
res=2
octaves = 15
persistence = 0.60
lacunarity = 2

ax[0].imshow(civ,cmap="Greens")
ax[0].set_title("Civilisation")
ax[0].scatter(x=[i[0] for i in a], y=[i[1] for i in a],color="Black")

ax[1].imshow(ter,cmap="Accent")
ax[1].set_title("Terrain")
ax[1].scatter(x=[i[0] for i in a], y=[i[1] for i in a],color="Black")


ax[2].imshow(thr,cmap="Reds")
ax[2].set_title("Threat")
ax[2].scatter(x=[i[0] for i in a], y=[i[1] for i in a],color="Black")


ax[3].imshow(fac,cmap="Greys")
ax[3].set_title("Factions")
ax[3].scatter(x=[i[0] for i in a], y=[i[1] for i in a],color="Black")



In [ ]:
def ar_xo(parent_1,parent_2):

    """
    Takes a "pack" of world encounters, selects two random crossing points from the list of encounters and uses weighted averages to randomize each points.
    """

    alpha = random.random()

    parent_1_coor = [i.coord for i in parent_1.pack] 
    parent_2_coor = [i.coord for i in parent_2.pack]

    offspring_1 = copy.deepcopy(parent_1)
    offspring_2 = copy.deepcopy(parent_2)
    
    cut_points = [random.randint(0,len(parent_1_coor))-1]

    second_cut = abs(random.randint(0,len(parent_1_coor)-1))
    
    while second_cut == cut_points[0]:
        second_cut = abs(random.randint(0,len(parent_1_coor)-1))
    
    cut_points.append(second_cut)
    cut_points = sorted(cut_points)

    new_coord_1 = parent_1_coor.copy()
    new_coord_2 = parent_2_coor.copy()

    for i in range(cut_points[0],cut_points[1]+1):
    
    # Takes X and Y coordinates and gets the weighted average for each point using alpha value.
    
        new_coord_1[i] = [((new_coord_1[i][0]*alpha) + (new_coord_2[i][0]*(1-alpha))), ((new_coord_1[i][1]*alpha) + (new_coord_2[i][1]*(1-alpha)))]
        new_coord_2[i] = [((new_coord_2[i][0]*alpha) + (new_coord_1[i][0]*(1-alpha))), ((new_coord_2[i][1]*alpha) + (new_coord_1[i][1]*(1-alpha)))]

    offspring_1.pack_coord = new_coord_1
    offspring_2.pack_coord = new_coord_2

    offspring_1.update_pack_coord()
    offspring_2.update_pack_coord()
    offspring_1.update_pack_fitness()
    offspring_2.update_pack_fitness()


    return offspring_1,offspring_2


In [ ]:
a = pop.population[0]
b = pop.population[1]

In [ ]:
for idx, i in enumerate(ar_xo(a,b)[0]):
    print(a.pack_coord[idx] == i)